## 

In [115]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath('..')))

In [116]:
import pandas as pd
from scripts.database import DbConn, DatabaseConn

In [117]:
detected_image = pd.read_csv('../image/final_results/detected_images.csv')

In [118]:
detected_image.head()

,Image,xmin,ymin,xmax,ymax,confidence,class
0,@lobelia4cosmetics_11062.jpg,183.597168,0.000000,1111.034180,1264.329590,0.976964,Foodpackage
1,@lobelia4cosmetics_11061.jpg,242.018066,0.000000,1087.031128,1280.000000,0.668643,Lotion_Moisturizers
2,@lobelia4cosmetics_11060.jpg,98.514877,11.680565,701.334839,606.288147,0.974131,Lotion_Moisturizers
3,@lobelia4cosmetics_11059.jpg,167.451294,0.384521,861.117920,1278.018555,0.897209,Lotion_Moisturizers
4,@lobelia4cosmetics_11058.jpg,420.406372,115.573479,686.156494,825.498718,0.952333,Supplement


In [119]:
detected_image.shape

(3128, 7)

In [120]:
dbconn = DbConn()
db_data = dbconn.read_data('lobeliapha_transformed')

Connected to database


c:\Users\Hp\Desktop\Kifiya\week7\ETH-MedData-Warehouse\scripts\database.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {table_name}", self.conn)


In [121]:
db_data = pd.DataFrame(db_data, columns=['ID', 'date', 'product_name', 'weight', 'price', 'Telegram Address', 'address', 'Phone Number', 'Delivery Fee', 'Media Path'])
db_data.head()

,ID,date,product_name,weight,price,Telegram Address,address,Phone Number,Delivery Fee,Media Path
0,12401,2024-10-10 14:47:05,sulfur8,205g,3000,https://t.me/lobelia4cosmetics,adress:- infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12401.jpg
1,12380,2024-10-10 04:07:50,nido usa,2.2kg,5900,https://t.me/lobelia4cosmetics,adress:- infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12380.jpg
2,12378,2024-10-10 04:07:50,nido usa,2.2kg,5900,https://t.me/lobelia4cosmetics,adress:- infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12378.jpg
3,12367,2024-10-09 15:22:01,nido,1.8kg,4500,https://t.me/lobelia4cosmetics,adress:- infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12367.jpg
4,12366,2024-10-09 12:44:53,sulfur8,205g,3000,https://t.me/lobelia4cosmetics,adress:- infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12366.jpg


In [122]:
# A little preprocessing on address
db_data['address'] = db_data['address'].str.strip().str.replace('adress:- ', '')

In [123]:
db_data.head()

,ID,date,product_name,weight,price,Telegram Address,address,Phone Number,Delivery Fee,Media Path
0,12401,2024-10-10 14:47:05,sulfur8,205g,3000,https://t.me/lobelia4cosmetics,infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12401.jpg
1,12380,2024-10-10 04:07:50,nido usa,2.2kg,5900,https://t.me/lobelia4cosmetics,infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12380.jpg
2,12378,2024-10-10 04:07:50,nido usa,2.2kg,5900,https://t.me/lobelia4cosmetics,infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12378.jpg
3,12367,2024-10-09 15:22:01,nido,1.8kg,4500,https://t.me/lobelia4cosmetics,infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12367.jpg
4,12366,2024-10-09 12:44:53,sulfur8,205g,3000,https://t.me/lobelia4cosmetics,infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12366.jpg


In [124]:
# Extracting ID from the detection results table to create relation with other tables
detected_image['ID'] = detected_image['Image'].apply(lambda x: x.split('_')[1].split('.')[0])
detected_image.loc[:, 'ID'] = detected_image['ID'].astype(str)
detected_images = detected_image.drop_duplicates(subset=['ID'], keep='first')

In [125]:
detected_images.head()

,Image,xmin,ymin,xmax,ymax,confidence,class,ID
0,@lobelia4cosmetics_11062.jpg,183.597168,0.000000,1111.034180,1264.329590,0.976964,Foodpackage,11062
1,@lobelia4cosmetics_11061.jpg,242.018066,0.000000,1087.031128,1280.000000,0.668643,Lotion_Moisturizers,11061
2,@lobelia4cosmetics_11060.jpg,98.514877,11.680565,701.334839,606.288147,0.974131,Lotion_Moisturizers,11060
3,@lobelia4cosmetics_11059.jpg,167.451294,0.384521,861.117920,1278.018555,0.897209,Lotion_Moisturizers,11059
4,@lobelia4cosmetics_11058.jpg,420.406372,115.573479,686.156494,825.498718,0.952333,Supplement,11058


In [126]:
print(detected_images['ID'].dtype)  # Check data type of ID in detected_images
print(db_data['ID'].dtype)  # Check data type of ID in db_data
Ids = detected_images['ID'].values
print(type(Ids))  # Check data type of Ids

object
object
<class 'numpy.ndarray'>


In [127]:
product_cols = ['product_name', 'weight', 'price', 'address', 'Phone Number', 'ID']
telegram_cols = ['date', 'Telegram Address', 'Media Path', 'ID']

In [128]:
Products = db_data[product_cols][db_data['ID'].isin(detected_images['ID'].values)]
Products.set_index('ID', inplace=True)

In [129]:
Telegram = db_data[telegram_cols][db_data['ID'].isin(detected_images['ID'].values)]
Telegram.set_index('ID', inplace=True)

In [130]:
Ids = Products.index
detected_images = detected_images[detected_images['ID'].isin(Ids)]
detected_images.set_index('ID', inplace=True)

In [131]:
# Rename product table columns
Products.columns = ['Product', 'Prescription', 'Price', 'Address', 'PhoneNumber'] 
# Rename Telegram Table Columns
Telegram.columns = ['Date', 'Channel_username', 'Media_path']

In [132]:
detected_images.columns = ['Image', 'x_min', 'y_min', 'x_max', 'y_max', 'confidence', 'label']

In [133]:
Products.shape, detected_images.shape, Telegram.shape

((470, 5), (470, 7), (470, 3))

In [134]:
Products = Products.reset_index()
Telegram = Telegram.reset_index()
detected_images = detected_images.reset_index()

In [135]:
# Convert columns to desired types
Products['Product'] = Products['Product'].astype(str)
Products['Prescription'] = Products['Prescription'].astype(str)
Products['Price'] = Products['Price'].astype(int)
Products['Address'] = Products['Address'].astype(str)
Products['PhoneNumber'] = Products['PhoneNumber'].astype(str)
Products['ID'] = Products['ID'].astype(int)
Telegram['ID'] = Telegram['ID'].astype(int)
detected_images['ID'] = detected_images['ID'].astype(int)

In [137]:
detected_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          470 non-null    int64  
 1   Image       470 non-null    object 
 2   x_min       470 non-null    float64
 3   y_min       470 non-null    float64
 4   x_max       470 non-null    float64
 5   y_max       470 non-null    float64
 6   confidence  470 non-null    float64
 7   label       470 non-null    object 
dtypes: float64(5), int64(1), object(2)
memory usage: 29.5+ KB


In [136]:
databaseConn = DatabaseConn()
databaseConn.insert_dataframe_data('Products', Products)
databaseConn.insert_dataframe_data('Telegram', Telegram)
databaseConn.insert_dataframe_data('detected_images', detected_images)

Connected to database using SQLAlchemy
